<a href="https://colab.research.google.com/github/brandonliu12/nfl-qb-prediction/blob/main/Project_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries, read files, and clean it further

In [39]:
import pandas as pd
path = 'NFL Play by Play 2009-2018 (v5).csv'
df = pd.read_csv(path)

df['game_date'] = pd.to_datetime(df['game_date'])
df['year'] = df['game_date'].dt.year
df_2018 = df[df['game_date'].dt.year == 2018]
df = df[df['year'] != 2018]



<ipython-input-39-31ba2dc60365>:3: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [40]:
df_2018.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,year
407335,37,2018090600,PHI,ATL,ATL,away,PHI,PHI,35.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407336,52,2018090600,PHI,ATL,ATL,away,PHI,ATL,75.0,2018-09-06,...,L.Paulsen,5.0,0,NaN,False Start,0.0,0.0,0.0,0.0,2018
407337,75,2018090600,PHI,ATL,ATL,away,PHI,ATL,80.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407338,104,2018090600,PHI,ATL,ATL,away,PHI,ATL,70.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018
407339,125,2018090600,PHI,ATL,ATL,away,PHI,ATL,59.0,2018-09-06,...,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0,2018


In [41]:
brady_df = df[(df['home_team'] == 'NE') | (df['away_team'] == 'NE')]


In [42]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['passer_player_name_encoded'] = encoder.fit_transform(df['passer_player_name'])

In [43]:
import numpy as np
relevant_columns = ['yardline_100','game_seconds_remaining','qtr', 'down', 'air_yards', 'pass_attempt', 'posteam_timeouts_remaining',
                     'complete_pass', 'passer_player_name_encoded', 'pass_location', 'score_differential_post', 'posteam_type', 'half_seconds_remaining', 'play_type', 'ep', 'wp', 'no_score_prob', 'opp_fg_prob', 'opp_td_prob', 'fg_prob', 'safety_prob', 'td_prob']

# Select the data using the relevant columns
df_selected = df[relevant_columns]
df_selected['pass_location'].fillna('no_throw', inplace=True)


# Get list of categorical columns
categorical_cols = df_selected.select_dtypes(include=['object']).columns

# Create dummy variables for all categorical columns
df_dummies = pd.get_dummies(df_selected[categorical_cols])

# Combine original numerical data with dummy variables
df_combined = pd.concat([df_selected.select_dtypes(include=[np.number]), df_dummies], axis=1)

df_combined = df_combined.fillna(0)

df_combined.head()

<ipython-input-43-82595c0c10a9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['pass_location'].fillna('no_throw', inplace=True)


,yardline_100,game_seconds_remaining,qtr,down,air_yards,pass_attempt,posteam_timeouts_remaining,complete_pass,passer_player_name_encoded,score_differential_post,...,posteam_type_home,play_type_extra_point,play_type_field_goal,play_type_kickoff,play_type_no_play,play_type_pass,play_type_punt,play_type_qb_kneel,play_type_qb_spike,play_type_run
0,30.0,3600.0,1,0.0,0.0,0.0,3.0,0.0,346,0.0,...,1,0,0,1,0,0,0,0,0,0
1,58.0,3593.0,1,1.0,-3.0,1.0,3.0,1.0,47,0.0,...,1,0,0,0,0,1,0,0,0,0
2,53.0,3556.0,1,2.0,0.0,0.0,3.0,0.0,346,0.0,...,1,0,0,0,0,0,0,0,0,1
3,56.0,3515.0,1,3.0,34.0,1.0,3.0,0.0,47,0.0,...,1,0,0,0,0,1,0,0,0,0
4,56.0,3507.0,1,4.0,0.0,0.0,3.0,0.0,346,0.0,...,1,0,0,0,0,0,1,0,0,0


In [45]:
df_2018['passer_player_name_encoded'] = encoder.fit_transform(df_2018['passer_player_name'])


df_selected_2018 = df_2018[relevant_columns + ['play_id']]
df_selected_2018['pass_location'].fillna('no_throw', inplace=True)


# Get list of categorical columns
categorical_cols = df_selected_2018.select_dtypes(include=['object']).columns


# Create dummy variables for all categorical columns in 2018 dataframe
df_dummies_2018 = pd.get_dummies(df_selected_2018[categorical_cols])

# Combine original numerical data with dummy variables for 2018 dataframe
df_combined_2018 = pd.concat([df_selected_2018.select_dtypes(include=[np.number]), df_dummies_2018, df_selected_2018['play_id']], axis=1)

# Fill NaN values with 0 for 2018 dataframe
df_combined_2018 = df_combined_2018.fillna(0)
df_2018 = df_2018.fillna(0)


df_combined_2018.head()

<ipython-input-45-c0f073a50cc0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected_2018['pass_location'].fillna('no_throw', inplace=True)


,yardline_100,game_seconds_remaining,qtr,down,air_yards,pass_attempt,posteam_timeouts_remaining,complete_pass,passer_player_name_encoded,score_differential_post,...,play_type_extra_point,play_type_field_goal,play_type_kickoff,play_type_no_play,play_type_pass,play_type_punt,play_type_qb_kneel,play_type_qb_spike,play_type_run,play_id
407335,35.0,3600.0,1,0.0,0.0,0.0,3.0,0.0,98,0.0,...,0,0,1,0,0,0,0,0,0,37
407336,75.0,3600.0,1,1.0,0.0,0.0,3.0,0.0,98,0.0,...,0,0,0,1,0,0,0,0,0,52
407337,80.0,3600.0,1,1.0,8.0,1.0,3.0,1.0,69,0.0,...,0,0,0,0,1,0,0,0,0,75
407338,70.0,3562.0,1,2.0,0.0,0.0,3.0,0.0,98,0.0,...,0,0,0,0,0,0,0,0,1,104
407339,59.0,3526.0,1,1.0,0.0,0.0,3.0,0.0,98,0.0,...,0,0,0,0,0,0,0,0,1,125


Machine Learning Model

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Define target variable
target = df_combined['air_yards']

# Define features
features = df_combined.drop('air_yards', axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [47]:
from sklearn.ensemble import RandomForestRegressor

# Create a random forest regressor object
random_forest = RandomForestRegressor(random_state=42, n_estimators=100)

# Train the model using the training sets
random_forest.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = random_forest.predict(X_test)


In [48]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, y_pred))

print(f"Model RMSE: {rmse}")


Model RMSE: 5.928931314954662


In [49]:
mean_air_yards = df['air_yards'].mean()
mean_air_yards

8.345890389457095

Predict 2018 performance

In [50]:
# Separate the features from the target variable in df_2018
features_2018 = df_combined_2018.drop(['air_yards', 'play_id'], axis=1)

# Use the trained model to predict 'air_yards' for df_2018
predictions_2018 = random_forest.predict(features_2018)

# Add the predictions as a new column in df_2018
df_2018['predicted_air_yards'] = predictions_2018

<ipython-input-50-65e47ed5e1e5>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_2018['predicted_air_yards'] = predictions_2018


In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separate the actual 'air_yards' values
actuals_2018 = df_2018['air_yards']

# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(actuals_2018, predictions_2018)
mse = mean_squared_error(actuals_2018, predictions_2018)
rmse = np.sqrt(mse)  # or directly use mean_squared_error(actuals_2018, predictions_2018, squared=False)
r2 = r2_score(actuals_2018, predictions_2018)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")


MAE: 3.197909753091455
MSE: 36.14401528936618
RMSE: 6.011989295513273
R-squared: 0.3461315129521503


In [52]:
predicted_df = df_2018[['play_id','passer_player_name_encoded', 'yardline_100','game_seconds_remaining','qtr', 'down','air_yards', 'predicted_air_yards']].copy()

predicted_df['pass_distance_type'] = 'short'  # default value
predicted_df.loc[predicted_df['air_yards'] > 20, 'pass_distance_type'] = 'long'
predicted_df.loc[predicted_df['air_yards'] < 0, 'pass_distance_type'] = 'no_throw'

predicted_df['predicted_pass_distance_type'] = 'short'  # default value
predicted_df.loc[predicted_df['predicted_air_yards'] > 20, 'predicted_pass_distance_type'] = 'long'
predicted_df.loc[predicted_df['predicted_air_yards'] < 0, 'predicted_pass_distance_type'] = 'no_throw'

predicted_df.head()


,play_id,passer_player_name_encoded,yardline_100,game_seconds_remaining,qtr,down,air_yards,predicted_air_yards,pass_distance_type,predicted_pass_distance_type
407335,37,98,35.0,3600.0,1,0.0,0.0,1.22,short,short
407336,52,98,75.0,3600.0,1,1.0,0.0,-0.26,short,no_throw
407337,75,69,80.0,3600.0,1,1.0,8.0,6.92,short,short
407338,104,98,70.0,3562.0,1,2.0,0.0,0.30,short,short
407339,125,98,59.0,3526.0,1,1.0,0.0,0.37,short,short


In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score

# Assuming y_true and y_pred are your true and predicted labels respectively
y_true = predicted_df['pass_distance_type']
y_pred = predicted_df['predicted_pass_distance_type']

confusion_mat = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(confusion_mat)

accuracy = accuracy_score(y_true, y_pred)
print("Accuracy: ", accuracy)

Confusion Matrix:
[[  109     0  1613]
 [   14    12  2424]
 [  107  4072 33685]]
Accuracy:  0.8042154343895709


Continue predicting additional statistic such as pass location and distance

In [54]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.exceptions import NotFittedError

# Define target variable for pass location

target_pass_location = df_selected['pass_location']

# Define features for pass location
features_pass_location = df_combined.drop(['pass_location_left', 'pass_location_middle', 'pass_location_right', 'pass_location_no_throw'], axis=1)

# Split the data into train and test sets for pass location
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(features_pass_location, target_pass_location, test_size=0.2, random_state=42)

# Create a random forest classifier object
random_forest_classifier = RandomForestClassifier(random_state=42, n_estimators=100)

# Train the classifier model using the training sets
random_forest_classifier.fit(X_train_loc, y_train_loc)

# Make predictions using the testing set
y_pred_loc = random_forest_classifier.predict(X_test_loc)

# Compute Accuracy
accuracy = accuracy_score(y_test_loc, y_pred_loc)

print(f"Model Accuracy: {accuracy}")

# Separate the features from the target variable in df_2018 for pass location
features_2018_loc = df_combined_2018.drop(['pass_location_left', 'pass_location_middle', 'pass_location_right',  'pass_location_no_throw','play_id'], axis=1)

# Use the trained classifier to predict 'pass_location' for df_2018
predictions_2018_loc = random_forest_classifier.predict(features_2018_loc)

# Create a dataframe for predicted pass location
pass_location_predicted_df = df_2018[['play_id']].copy()
pass_location_predicted_df['pass_location'] = df_selected_2018['pass_location']
pass_location_predicted_df['predicted_pass_location'] = predictions_2018_loc

pass_location_predicted_df.head()

Model Accuracy: 0.7559257122515866


,play_id,pass_location,predicted_pass_location
407335,37,no_throw,no_throw
407336,52,no_throw,no_throw
407337,75,right,right
407338,104,no_throw,no_throw
407339,125,no_throw,no_throw


Merge with previous predictions

In [57]:
final_predicted_df = pd.merge(predicted_df, pass_location_predicted_df, how='outer', on='play_id')
final_predicted_df = final_predicted_df.drop_duplicates(subset='play_id')
final_predicted_df['passer_player_name'] = encoder.inverse_transform(final_predicted_df['passer_player_name_encoded'])
final_predicted_df.head()

,play_id,passer_player_name_encoded,yardline_100,game_seconds_remaining,qtr,down,air_yards,predicted_air_yards,pass_distance_type,predicted_pass_distance_type,pass_location,predicted_pass_location,passer_player_name
0,37,98,35.0,3600.0,1,0.0,0.0,1.22,short,short,no_throw,no_throw,NaN
256,52,98,75.0,3600.0,1,1.0,0.0,-0.26,short,no_throw,no_throw,no_throw,NaN
337,75,69,80.0,3600.0,1,1.0,8.0,6.92,short,short,right,right,M.Ryan
1013,104,98,70.0,3562.0,1,2.0,0.0,0.30,short,short,no_throw,no_throw,NaN
1182,125,98,59.0,3526.0,1,1.0,0.0,0.37,short,short,no_throw,no_throw,NaN


In [63]:
player_dict = dict(df[['passer_player_name_encoded', 'passer_player_name']].drop_duplicates().values)